# Producer

In [1]:
# imports
import pathlib, logging, importlib
from threading import Thread
from openmsitoolbox.logging import OpenMSILogger
from openmsistream import UploadDataFile, DataFileUploadDirectory

In [2]:
# Configure a logger (only needed when running in a Jupyter notebook like this)
logger = OpenMSILogger("LocalProducer", filelevel=None)
importlib.reload(logging)

<module 'logging' from '/Users/namanparikh/opt/anaconda3/envs/openmsi/lib/python3.9/logging/__init__.py'>

In [6]:
# The name of the topic to work with
TOPIC_NAME = "tutorial_data"

# Paths to the config file and the directory holding the test files
repo_root_dir = pathlib.Path().resolve().parent
CONFIG_FILE_PATH = repo_root_dir / "streaming_2" / "config_files" / "confluent_cloud_broker.config"
TEST_FILE_DIR = repo_root_dir / "streaming_2" / "test_folder"

## Upload

In [4]:
def upload_task(upload_directory, *args, **kwargs):
    """Run "upload_files_as_added" for a given DataFileUploadDirectory, and log a message
    when it gets shut down

    Args:
        upload_directory (DataFileUploadDirectory): the DataFileUploadDirectory to run
        args (list): passed through to "upload_files_as_added"
        kwargs (dict): passed through to "upload_files_as_added"
    """
    # This call to "upload_files_as_added" waits until the program is shut down
    uploaded_filepaths = upload_directory.upload_files_as_added(*args, **kwargs)
    msg = (
        f"The following files were uploaded:\n\t"
    )
    msg += "\n\t".join([str(fp) for fp in uploaded_filepaths])
    upload_directory.logger.info(msg)

In [7]:
# Create the DataFileUploadDirectory
dfud = DataFileUploadDirectory(TEST_FILE_DIR, CONFIG_FILE_PATH, logger=logger)
# Start running its "upload_files_as_added" function in a separate thread
upload_thread = Thread(
    target=upload_task,
    args=(
        dfud,
        TOPIC_NAME,
    ),
)
upload_thread.start()

[LocalProducer 2024-07-18 15:23:18] Will upload new files added to/Users/namanparikh/Documents/GitHub/paradim/reu2024-hyperspectral-camera/streaming_2/test_folder to the tutorial_data topic as 524288-byte chunks using 2 threads


In [ ]:
# Manually shut down the upload directory (if running from the command line this would
# be like typing "q" in the Terminal window)
dfud.control_command_queue.put("q")
upload_thread.join()